In [ ]:
%cd /Users/idawatibustan/Dev/shopee_data_science
%env PROJECT_PATH /Users/idawatibustan/Dev/shopee_data_science

In [ ]:
from utils.envs import *
from pprint import pprint
import pandas as pd
import numpy as np
import json

pd.set_option('display.max_colwidth', 100)

In [ ]:
fashion_train = pd.read_csv(fashion_train_repo)
fashion_val = pd.read_csv(fashion_val_repo)
fashion_train.head()

In [ ]:
fashion_profile = json.load(open(fashion_profile_json))
fashion_profile.keys()

In [ ]:
from functools import reduce

fashion_max_length = {}
for feature, library in fashion_profile.items():
    lengths = [len(x.split()) for x in library.keys()]
    fashion_max_length[feature] = max(lengths)
fashion_max_length

## Building Libraries of Secondary/ Tertiary Comparison 

In [ ]:
fashion_profile_secondary={}

### Pattern

In [ ]:
feature = 'Pattern'
lib = {}

val = fashion_profile[feature].get('plain')
print(val)
lib['polos'] = val

val = fashion_profile[feature].get('brocade')
print(val)
lib['brokat'] = val

fashion_profile_secondary[feature] = lib

### Collar Type

In [ ]:
feature = 'Collar Type'
lib = {}

val = fashion_profile[feature].get('off the shoulder')
print(val)
fashion_profile[feature].update({'off shoulder': val})
lib['off bahu'] = val
lib['tanpa bahu'] = val
# lib['slash neck'] = val
# lib['cold shoulder'] = val

val = fashion_profile[feature].get('o neck')
print(val)
lib['leher o'] = val
lib['kerah o'] = val
lib['crew neck'] = val
lib['round neck'] = val

val = fashion_profile[feature].get('v neck')
print(val)
lib['leher v'] = val
lib['kerah v'] = val
lib['tops v'] = val
lib['neck v'] = val
lib['v collar'] = val
lib['deep v'] = val

val = fashion_profile[feature].get('square neck')
print(val)
lib['square'] = val

val = fashion_profile[feature].get('hooded')
print(val)
lib['hoodie'] = val
lib['hoodies'] = val

val = fashion_profile[feature].get('shawl collar')
print(val)
lib['syal'] = val
lib['scarf'] = val
lib['shawl'] = val

fashion_profile_secondary[feature] = lib

### Fashion Trend

In [ ]:
feature = 'Fashion Trend'
lib = {}

val = fashion_profile[feature].get('party')
print(val)
fashion_profile[feature].update({'pesta': val})

val = fashion_profile[feature].get('retro vintage')
print(val)
lib['retro'] = val
fashion_profile[feature].update({'vintage': val})
# lib['vintage'] = val

val = fashion_profile[feature].get('korean')
print(val)
# lib['korea'] = val

val = fashion_profile[feature].get('office')
print(val)
lib['kantor'] = val


fashion_profile_secondary[feature] = lib

### Clothing Material

In [ ]:
feature = 'Clothing Material'
lib = {}

val = fashion_profile[feature].get('cotton')
print(val)
# lib['katun'] = val

val = fashion_profile[feature].get('chiffon')
print(val)
lib['sifon'] = val


fashion_profile_secondary[feature] = lib

### Sleeves

In [ ]:
feature = 'Sleeves'
lib = {}

val = fashion_profile[feature].get('long sleeve')
print(val)
fashion_profile[feature].update({'lengan panjang': val})
# lib['lengan panjang'] = val


val = fashion_profile[feature].get('short sleeve')
print(val)
fashion_profile[feature].update({'lengan pendek': val})
# lib['lengan pendek'] = val

val = fashion_profile[feature].get('sleeveless')
print(val)
fashion_profile[feature].update({'tanpa lengan': val})
# lib['tanpa lengan'] = val
lib['tali strap'] = val

val = fashion_profile[feature].get('sleeve 3 4')
print(val)
fashion_profile[feature].update({'lengan 3 4': val})
# lib['lengan flare 3 4'] = val
lib['3 4'] = val


fashion_profile_secondary[feature] = lib

## Storing Library

In [ ]:
fashion_library = {
    "primary": fashion_profile,
    "secondary": fashion_profile_secondary,
    "length": fashion_max_length,
}
with open('data/fashion_library.json', 'w+') as file:
    json.dump(fashion_library, file)

## heuristic functions

In [ ]:
""" heuristic function :) """
def _word_compare(texts_lib, library):
    for text in texts_lib:
        value = library.get(text, None)
        if not value is None:
            return value

def word_compare(title, libraries, level=4):
    texts = title.split()
    texts_lib = texts
    texts_temp = texts
    for i in range(1, level):
        texts_temp = list(zip(texts_temp, texts[i:]))
        texts_temp = [' '.join(text) for text in texts_temp]
        texts_lib += texts_temp

    if isinstance(libraries, list):
        for lib in libraries:
            value = _word_compare(texts_lib, lib)
            if not value is None:
                return value
        return None
    else:
        return _word_compare(texts_lib, libraries)

    
def get_feature(title, library, level=3):
    res = word_compare(
        title,
        libraries=library,
        level=level
    )
    return res


In [ ]:
""" get text details of all the features"""
fashion_lib = {}

for feature in fashion_profile.keys():
    key_to_text = {v: k for k, v in fashion_profile.get(feature).items()}
    fashion_lib[feature] = key_to_text
    fashion_train[feature+'_info'] = fashion_train[feature].map(key_to_text)


## Run heuristic function 

In [ ]:
""" run single lib heuristic """
from datetime import datetime
start_time = datetime.now()

fashion_pred = fashion_train.filter(['itemid', 'title'])
for feature in fashion_profile.keys():

    libraries = [fashion_profile[feature], fashion_profile_secondary[feature]]
    level = fashion_max_length[feature]

    fashion_pred[feature] = fashion_train.title.apply(get_feature, args=(libraries,level,))

duration = datetime.now() - start_time
print(duration)

fashion_pred.head()

In [ ]:
""" single feature prediction"""
start_time = datetime.now()

feature = 'Fashion Trend'

libraries = [fashion_profile[feature], fashion_profile_secondary[feature]]
level = fashion_max_length[feature]

fashion_pred[feature] = fashion_train.title.apply(get_feature, args=(libraries,level,))

duration = datetime.now() - start_time
print(duration)

## Review result

In [ ]:
result = {}
for feature in fashion_profile.keys():
    print(feature)
    """ compare actual brand and prediction on insight DF"""
    insight = fashion_train.filter(['itemid', 'title', feature, feature+'_info'])
    insight[feature+'_pred'] = fashion_pred[feature]
    insight[feature+'_info_pred'] = fashion_pred[feature].map(fashion_lib[feature])
    total = len(insight)

    """ drop brand NA of actual brand"""
    insight = insight.dropna(subset=[feature])
    print("  DF  ~NaN len:", len(insight), '({})%'.format(round(len(insight)/total*100)))
    # print(insight[feature].eq(insight[feature+'_pred']).value_counts(normalize=True))

    insight_accuracy = insight.dropna(subset=[feature, feature+'_pred'])
    print("  DF known len:",len(insight_accuracy), '({})%'.format(round(len(insight_accuracy)/len(insight)*100)))
    print(insight_accuracy[feature].eq(insight_accuracy[feature+'_pred']).value_counts(normalize=True))

#     """ get unequal insight_error DF"""
#     insight_error = insight.loc[insight[feature] != insight[feature+'_pred'], :]
#     insight_error[feature].value_counts()

    """ get unequal insight_error DF"""
    insight_error = insight_accuracy.loc[insight_accuracy[feature] != insight_accuracy[feature+'_pred'], :]
    print(insight_error[feature].value_counts())

    result[feature] = insight
    print()

In [ ]:
""" view top unknown/ error """
idx = 3
feature = list(fashion_profile.keys())[idx]
insight = result[feature]

res = insight[feature].eq(insight[feature+'_pred'])
wrong_index = res[res == False].index
insight_error = insight.loc[wrong_index, :]
print(insight_error[feature].value_counts().head())

print( fashion_lib[feature].get(7) )
print( fashion_lib[feature].get(18) )
print( fashion_lib[feature].get(3) )
print( fashion_lib[feature].get(4) )
print( fashion_lib[feature].get(16) )

In [ ]:
""" view error title for : bkey """
# feature = 'Pattern'
bkey = 3

print(fashion_lib[feature].get(bkey))
feature_error = insight_error.loc[insight_error[feature] == bkey,:]
print(len(feature_error))

feature_error.dropna(subset=[feature+'_pred'])

# feature_error.loc[feature_error[feature+'_pred'].isna(), :].title

# for t in feature_error.loc[feature_error[feature+'_pred'].isna(), :].title:
#     print(t)

# print(feature_error[feature+'_pred'].value_counts())
# print(fashion_lib[feature].get(1))
# feature_error.loc[feature_error[feature+'_pred'] == 1, :].title

## Run heuristics and export submission

In [ ]:
""" fashion_val, run all heuristic functions - prediction on separate df """
from datetime import datetime
start_time = datetime.now()

fashion_val_pred = fashion_val.filter(['itemid', 'title'])
for feature in fashion_profile.keys():
    libraries = [fashion_profile[feature], fashion_profile_secondary[feature]]
    level = fashion_max_length[feature]

    fashion_val_pred[feature] = fashion_val.title.apply(get_feature, args=(libraries,level,))

duration = datetime.now() - start_time
print(duration)

fashion_val_pred

In [ ]:
# format answer
fashion_val_submission = pd.DataFrame(columns=['id', 'tagging'])

temp = pd.DataFrame()
for feature in fashion_profile.keys():
    temp['id'] = fashion_val_pred['itemid'].map(str)+"_"+feature
    temp['tagging'] = fashion_val_pred[feature]
    
    if len(fashion_val_submission) == 0:
        fashion_val_submission = temp
    else:
        fashion_val_submission = fashion_val_submission.append(temp, ignore_index=True)
    print(feature, len(fashion_val_submission))
    
        
# fashion_submission.append(temp, ignore_index=True)
fashion_val_submission.head()
# fashion_val_submission.info()
# temp

In [ ]:
fashion_val_submission.to_csv('fashion_info_val_submission.csv')